In [34]:
# Step 0: Install and import the libraries

In [5]:
!pip install numpy
!pip install gym

In [6]:
import numpy as np
import gym
import random

In [7]:
# Step 1: Create the environment


In [32]:
env = gym.make("Taxi-v3")
env.reset()
env.render()

In [11]:
# Step 2: Create the Q-table and initialize it

In [17]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")
action_space = env.action_space.n
print("There are ", action_space, " possible actions") # up, down, right, down, take-on, take-off

There are  500  possible states
There are  6  possible actions


In [14]:
# Create our Q table with state_size rows and action_size columns (500x6)
Q = np.zeros((state_space, action_space))
print(Q)
print(Q.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
(500, 6)


In [19]:
# Step 3: Define the hyperparameters

In [18]:
total_episodes = 25000        # Total number of training episodes (The more, the better)
total_test_episodes = 10      # Total number of test episodes (just for show to us)
max_steps = 200               # Max steps per episode (after 200 step the episode close)

learning_rate = 0.01          # Learning rate
gamma = 0.99                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.001           # Minimum exploration probability
decay_rate = 0.01             # Exponential decay rate for exploration prob

In [21]:
# Step 4: Define the epsilon-greedy policy

In [22]:
def epsilon_greedy_policy(Q, state): # Q = score of now  state = what state do we be
  # if random number > greater than epsilon --> exploitation (greedy)
  if(random.uniform(0,1) > epsilon):
    action = np.argmax(Q[state]) # maximum action
  # else --> exploration (random)
  else:
    action = env.action_space.sample()

  return action

In [20]:
# Step 5: Define the Q-Learning algorithm and train our agent

In [24]:
for episode in range(total_episodes): # in each episode ...
    # Reset the environment
    state = env.reset()
    step = 0
    done = False

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

    for step in range(max_steps): # 200 step is max
        #
        action = epsilon_greedy_policy(Q, state) # select action

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action) # it is a options of this library and we can create by our selves

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]     Update and Learning
        Q[state][action] = Q[state][action] + learning_rate * (reward + gamma * np.max(Q[new_state]) - Q[state][action])
        # If done : finish episode
        if done == True:
            break

        # Our new state is state
        state = new_state

In [25]:
# Step 6: Let's watch our autonomous

In [31]:
import time
rewards = []

frames = []
for episode in range(total_test_episodes): # create episode for show to us
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    print("****************************************************")
    print("EPISODE ", episode)
    for step in range(max_steps):
        env.render() # show enviroment
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(Q[state][:]) # we don't select action with e-greedy because we learning before and we take action just from Q-Table
        new_state, reward, done, info = env.step(action)
        total_rewards += reward

        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

****************************************************
EPISODE  0
****************************************************
EPISODE  1
****************************************************
EPISODE  2
****************************************************
EPISODE  3
****************************************************
EPISODE  4
****************************************************
EPISODE  5
****************************************************
EPISODE  6
****************************************************
EPISODE  7
****************************************************
EPISODE  8
****************************************************
EPISODE  9
Score over time: 8.0


In [33]:
Q

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-2.98146739, -3.04552473, -3.34395812, -3.04631192,  9.6220697 ,
        -3.33142803],
       [-1.45027051, -1.29697924, -1.48309371, -1.15478664, 14.11880599,
        -1.32393218],
       ...,
       [-0.72675663, -0.61750394, -0.74625407, -0.72568389, -0.77401061,
        -0.97859423],
       [-2.23155149, -2.22898763, -2.25532573, -1.45321135, -2.67444219,
        -2.6730212 ],
       [-0.05084896, -0.0260451 , -0.0199    ,  2.40152549, -0.28749926,
        -0.46353213]])